In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [25]:
# Import necessary libraries
import numpy as np
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms import VQC
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal

# Define your quantum feature map
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Define your variational form
ansatz = TwoLocal(2, ['ry', 'rz'], 'cz', reps=3)


# Define your quantum instance
quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))

# Define your classical optimizer
optimizer = COBYLA(maxiter=100)

# Define your quantum machine learning model
num_qubits = 2
model = VQC(num_qubits=num_qubits, optimizer=optimizer, feature_map=feature_map, ansatz=ansatz, quantum_instance=quantum_instance)
# Define your training data
training_data = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
labels = np.array([0, 1, 2])  # Map "Not Effective" to 0, "Effective" to 1, and "Highly Effective" to 2

# Train the model
model.fit(training_data, labels)

# Define new data for prediction
new_data = np.array([[0.31, 0.45]])

# Make predictions on new data
predicted_labels = model.predict(new_data)
predicted_labels = np.where(predicted_labels == 0, "Not Effective", np.where(predicted_labels == 1, "Effective", "Highly Effective"))



print('Predicted Labels:', predicted_labels)

/tmp/ipykernel_61/1755479642.py:18: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Predicted Labels: Effective


In [22]:
from qiskit import transpile

# Transpile the quantum circuit representing the model
circuit = transpile(model.circuit, quantum_instance.backend)

# Draw the transpiled circuit
print(circuit)

     ┌───┐┌─────────────┐                                          ┌───┐»
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■──┤ H ├»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├───┤»
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ H ├»
     └───┘└─────────────┘└───┘└──────────────────────────────┘└───┘└───┘»
«     ┌─────────────┐                                          ┌──────────┐»
«q_0: ┤ P(2.0*x[0]) ├──■────────────────────────────────────■──┤ Ry(θ[0]) ├»
«     ├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├──────────┤»
«q_1: ┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ Ry(θ[1]) ├»
«     └─────────────┘└───┘└──────────────────────────────┘└───┘└──────────┘»
«     ┌──────────┐   ┌──────────┐┌──────────┐   ┌──────────┐┌───────────┐   »
«q_0: ┤ Rz(θ[2]) ├─■─┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├─■─┤ Ry(θ[8]) ├┤ Rz(θ[10]) ├─■─»
«     ├──────────┤ │ ├──────────┤├──────────┤ │ ├──────────┤├───────────┤ │ »
«q_1: ┤ Rz(